<a href="https://colab.research.google.com/github/FabioRovai/RAFT/blob/main/Policy_Hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Questions we are trying to answer  
1. Type of requests by country  
2. Number of requests by country  
3. Trend over time of notices by country breakdown by # and type of request 

In [1]:
# Generalizing enviroment
!wget https://www.dropbox.com/s/74o2439vl4f705d/internet_freedom_scores.csv --quiet
!pip install gdown --q
!pip install pdfminer --q
!pip install pdfminer.six --q
!pip install striprtf --q

In [2]:
import gdown
import os
import shutil
import zipfile
import os
import pandas as pd
from striprtf.striprtf import rtf_to_text
from pdfminer.high_level import extract_text

In [3]:
'''url = 'https://drive.google.com/u/0/uc?export=download&confirm=dxYX&id=1Did693v60aQsIy3y02jYopLqQ9dX3T13'
output = 'goverement_requests.zip'
gdown.download(url, output, quiet=True)'''

"url = 'https://drive.google.com/u/0/uc?export=download&confirm=dxYX&id=1Did693v60aQsIy3y02jYopLqQ9dX3T13'\noutput = 'goverement_requests.zip'\ngdown.download(url, output, quiet=True)"

In [4]:
url = 'https://drive.google.com/u/0/uc?export=download&confirm=uAxG&id=1pw3S6t8bkSk0__RxCPfGSWwhsZuC9FDy'
output = 'notices.json'
gdown.download(url, output, quiet=True)

'notices.json'

In [5]:
url = 'https://drive.google.com/u/0/uc?id=10H32wf70nBFdwlqKvU_7my3BBPc8LDuv&export=download'
output = 'csv.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=10H32wf70nBFdwlqKvU_7my3BBPc8LDuv&export=download
To: /content/csv.csv
100%|██████████| 54.6M/54.6M [00:00<00:00, 184MB/s]


'csv.csv'

In [6]:
url = 'https://drive.google.com/u/0/uc?export=download&confirm=zN00&id=1yNIJLQWw6ZdQPrCYVFj-2P-i996L7CEj'
output = 'rtf.csv'
gdown.download(url, output, quiet=True)

'rtf.csv'

In [7]:
!mkdir goverement_requests
!mkdir Hack

In [ ]:
#Storing ZIP and unzip in 2 separated folders
shutil.move("/content/goverement_requests.zip", "/content/goverement_requests")

'/content/goverement_requests/goverement_requests.zip'

In [ ]:

with zipfile.ZipFile('/content/goverement_requests/goverement_requests.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Hack') 

In [8]:
import json

path = "/content/notices.json"

f = open(path ,encoding="utf8")

data = json.loads(f.read())
##Preview of one notice in JSON format
data[15643]


{'notice': {'action_taken': 'Yes',
  'body': '',
  'date_received': '2019-01-15T00:00:00.000Z',
  'date_sent': '2019-01-15T00:00:00.000Z',
  'id': 17906849,
  'jurisdictions': [],
  'language': '',
  'principal_name': 'Twitter',
  'recipient_name': 'Twitter',
  'sender_name': 'Federal Service for Supervision in the Sphere of Telecom, Information Technologies and Mass Communications',
  'tags': [],
  'title': 'Legal Request to TWITTER from Russia - ROSKOMNADZOR',
  'topics': ['Government Requests'],
  'type': 'GovernmentRequest',
  'works': [{'copyrighted_urls': [{'count': 1, 'domain': 'seeattachedPDF'}],
    'description': '',
    'infringing_urls': [{'count': 1, 'domain': 'seeattachedPDF'}]}]}}

**To do for Fabio, look into indexing to access elements in JSON

In [ ]:
print(data[0]['notice']['recipient_name']) #number index + main title (notice)+ section you like
print(data[10]['notice']['sender_name'])

Google, Inc.
Federal Service for Supervision of Communications, Information Technology and Mass Media (Roskomnadzor)


In [9]:
import pandas as pd
df=pd.json_normalize(data)


In [ ]:
df.to_csv('jsn.csv')



In [10]:
df.head()

,notice.id,notice.type,notice.title,notice.body,notice.date_sent,notice.date_received,notice.topics,notice.sender_name,notice.principal_name,notice.recipient_name,notice.works,notice.tags,notice.jurisdictions,notice.action_taken,notice.language
0,11062657,GovernmentRequest,Local Law Request to Google to Remove Illegal ...,\r\n\r\nRequest made in application of art. 12...,2015-06-22T00:00:00.000Z,2015-06-22T00:00:00.000Z,"[Government Requests, International]",L’Office Central de Lutte contre la Criminali...,None,"Google, Inc.",[{'description': ' Google is unable to publis...,[],[],,fr
1,11210026,GovernmentRequest,Address Confidentiality Program Participant,Members of the Address Confidentiality Program...,2015-09-16T00:00:00.000Z,2015-09-16T00:00:00.000Z,"[Court Orders, Government Requests]","Department of Justice, State of Oregon, Addres...",Rozlyn,Darlene Wease,[{'description': 'Remove search engine listing...,[newspaper posting(s)],[us],Partial,en
2,11214903,GovernmentRequest,Russia - Roskomnadzor Request to Twitter - Cou...,,2015-09-08T00:00:00.000Z,2015-09-08T00:00:00.000Z,"[International, Government Requests]",Federal Service for Supervision of Communicati...,N/A,"Twitter, Inc.","[{'description': '', 'infringing_urls': [{'dom...",[],[],Yes,
3,11348794,GovernmentRequest,Turkey - APU Request to Twitter - Country With...,,2015-10-22T00:00:00.000Z,2015-10-22T00:00:00.000Z,"[International, Government Requests]","APU's Similar Content Decision 2015-818, 2015...",NA,"Twitter, Inc.","[{'description': '', 'infringing_urls': [{'dom...",[],[],Yes,
4,11354689,GovernmentRequest,Photo,Enlever tout sir google perci \r\n\r\n\r\nMela...,2015-10-25T00:00:00.000Z,2015-10-25T00:00:00.000Z,"[Copyright, Copyright and Fair Use, Government...",Mer,Mer,Ri,"[{'description': '', 'infringing_urls': [{'url...",[image],[Je],Yes,fr


In [ ]:
len(df)

62729

In [76]:
g=df['notice.jurisdictions'].value_counts()
g.to_csv('bb.csv')

bb=pd.read_csv('bb.csv', header=None)
bb=bb.replace("['RU']", 'Russia')
bb=bb.replace("['ID']", 'Indonesia')
bb=bb.replace("['KR']", 'South Korea')
bb=bb.replace("['FR']", 'France')
bb=bb.replace("['TR']", 'Turkey')
bb=bb.replace("['IN']", 'India')
bb=bb.replace("['india']", 'India')
bb=bb.replace("['PK']", 'Pakistan')
bb=bb.replace("['GB']", 'United Kingdom')
bb=bb.replace("['DE']", 'Germany')
bb=bb.replace("['AU']", 'Australia')
bb=bb.replace("['it']", 'Italy')
bb=bb.replace("['Russia']", 'Russia')
bb=bb.replace("['ES']", 'Spain')
bb=bb.replace("['BE']", 'Belgium')
bb=bb.replace("['MY']", 'Malaysia')
bb=bb.replace("['SG']", 'Singapore')
bb=bb.replace("['IL']", 'Israel')

new_header = bb.iloc[0] #grab the first row for the header
bb = bb[1:] #take the data less the header row
bb.columns = new_header
bb.rename(columns={list(bb)[0]:  'Country'}, inplace=True)
bb.groupby(['Country', 'notice.jurisdictions']).sum().reset_index()
bbb=bb.iloc[:19]


In [99]:
ccc=pd.read_csv('/content/internet_freedom_scores.csv')
aaa=pd.merge(bbb, ccc, on="Country")
aaa.to_csv('melted.csv')



In [ ]:
BZZ=df['notice.jurisdictions'].value_counts()
BZZ.to_csv('number.csv')

In [ ]:
## Preview of one file
path = "/content/Hack"
rtf = open(path +"/17629538/notification_157232.rtf").read()
text = rtf_to_text(rtf)
#with pdf
g=('/content/Hack/11214903/201509041234567.pdf')
gx=extract_text(g)

print (text)
print (gx)




Import all rtfs and create a dataframe,

In [ ]:
'''##Create dataframe from notices
notices = []
ids = []
problems = []
for subdir, dirs, files in os.walk(path):
    for filename in files:
        filepath = subdir + os.sep + filename

        if filepath.endswith(".rtf"):
            try:
                rtf = open(filepath, encoding = "utf-8").read()
                text = rtf_to_text(rtf) 
                ids.append(subdir)
                notices.append(text)
            except (UnicodeDecodeError, UnicodeEncodeError, AttributeError) as e:
                problems.append(filepath)

d = {'ID':ids,'Notices':notices}
df = pd.DataFrame(d)
df.to_csv('rtf.csv')'''

In [ ]:
#some of the pdf are images really

In [ ]:
'''

notices = []
ids = []
problems = []
for subdir, dirs, files in os.walk(path):
    for filename in files:
        filepath = subdir + os.sep + filename

        if filepath.endswith(".pdf"):
          pdf = (filepath)
          text = extract_text(pdf) 
          ids.append(subdir[-8:])#no idea of what is doing this line
          notices.append(text)
          

foo = {'ID':ids,'Notices':notices}
df2 = pd.DataFrame(foo)
df2.to_csv('csv.csv')'''

**To do for Fabio, import all pdfs as well and append to the dataframe we have, you can use the outerloop just changing the if statement to have .endwith(".pdf") and then using the library pdfminer (https://pypi.org/project/pdfminer/), I didn't have time to try this. Don't spend too much time on it, you can move onto NLP with what we have, I can try to figure out importing pdf tomorrow if needed.

The ID column in df correspond to the ID in the JSON file